# 06 Rolling Regressions and Stability

Rolling windows to see regime changes.


## Table of Contents
- [Rolling regression](#rolling-regression)
- [Coefficient drift](#coefficient-drift)
- [Regime interpretation](#regime-interpretation)
- [Checkpoint (Self-Check)](#checkpoint-self-check)
- [Solutions (Reference)](#solutions-reference)


## Why This Notebook Matters
Regression is the bridge between statistics and ML. You will learn:
- single-factor vs multi-factor interpretation,
- robust standard errors,
- coefficient stability and multicollinearity.


## What You Will Produce
- (no file output; learning/analysis notebook)

## Success Criteria
- You can explain what you built and why each step exists.
- You can run your work end-to-end without undefined variables.

## Common Pitfalls
- Running cells top-to-bottom without reading the instructions.
- Leaving `...` placeholders in code cells.
- Treating coefficients as causal without a causal design.
- Ignoring multicollinearity (unstable coefficients).

## Matching Guide
- `docs/guides/02_regression/06_rolling_regressions_stability.md`



## How To Use This Notebook
- This notebook is hands-on. Most code cells are incomplete on purpose.
- Complete each TODO, then run the cell.
- Use the matching guide (`docs/guides/02_regression/06_rolling_regressions_stability.md`) for deep explanations and alternative examples.
- Write short interpretation notes as you go (what changed, why it matters).



<a id="environment-bootstrap"></a>
## Environment Bootstrap
Run this cell first. It makes the repo importable and defines common directories.



In [ ]:
from __future__ import annotations

from pathlib import Path
import sys


def find_repo_root(start: Path) -> Path:
    p = start
    for _ in range(8):
        if (p / 'src').exists() and (p / 'docs').exists():
            return p
        p = p.parent
    raise RuntimeError('Could not find repo root. Start Jupyter from the repo root.')


PROJECT_ROOT = find_repo_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
SAMPLE_DIR = DATA_DIR / 'sample'

PROJECT_ROOT



## Goal
Use rolling regressions to see how relationships change over time.

This is a realism check:
- A coefficient that is stable across decades is rare in macro.
- If coefficients drift, you should be cautious about "the" relationship.



## Primer: pandas Time Series Essentials

### DatetimeIndex
Most time series work in pandas assumes your DataFrame is indexed by time.

```python
import pandas as pd

# idx = pd.to_datetime(df['date'])
# df = df.set_index(idx).sort_index()
# assert isinstance(df.index, pd.DatetimeIndex)
```

If you see weird behavior (resample errors, merges not aligning), check:
- `df.index.dtype`
- `df.index.min(), df.index.max()`
- `df.index.is_monotonic_increasing`

### Resampling
> **Goal:** convert one frequency to another.

Common examples:
- daily -> month-end
- monthly -> quarter-end

```python
# month-end last value
# df_me_last = df.resample('ME').last()

# month-end mean
# df_me_mean = df.resample('ME').mean()

# quarter-end mean
# df_q_mean = df.resample('QE').mean()
```

**Interpretation matters.** For economic series:
- using `.last()` treats the end-of-period value as “the period’s value”
- using `.mean()` treats the period average as “the period’s value”

Neither is universally correct; you should choose based on measurement and use case.

### Alignment and merging
When joining multiple time series, you need to ensure they share:
- the same index type (`DatetimeIndex`)
- the same frequency convention (month-end vs month-start; quarter-end vs quarter-start)

```python
# Example: join two series and inspect missingness
# df = df1.join(df2, how='outer').sort_index()
# print(df.isna().sum())
```

### Lags and rolling windows
> **Lag:** use past values as features.

```python
# lag 1 period
# df['x_lag1'] = df['x'].shift(1)

# rolling mean (past-only)
# df['x_roll12'] = df['x'].rolling(12).mean()
```

### Common gotchas
- `shift(-1)` uses the future (leakage for forecasting).
- `rolling(..., center=True)` uses future values.
- Always `dropna()` after creating lags/rolls to get clean modeling rows.

One more gotcha:
- If you resample daily -> monthly and then create lags, your lag is “one month” (not one day). Lags are measured in the current index frequency.


## Primer: statsmodels vs scikit-learn (Inference vs Prediction)

### Two different goals
- **Prediction (ML):** optimize out-of-sample accuracy.
- **Inference (econometrics):** interpret coefficients + quantify uncertainty.

scikit-learn is mostly prediction-focused.
statsmodels is built for inference (standard errors, p-values, confidence intervals).

### Minimal statsmodels OLS pattern
```python
import statsmodels.api as sm

# X: DataFrame of features
# y: Series target

# Add intercept column
# Xc = sm.add_constant(X, has_constant='add')

# Fit OLS
# res = sm.OLS(y, Xc).fit()
# print(res.summary())
```

### What you're looking at in `res.summary()`
Typical regression output includes:
- **coef**: estimated coefficients $\hat\beta$
- **std err**: estimated standard errors $\widehat{SE}(\hat\beta)$
- **t**: t-statistic $\hat\beta / \widehat{SE}(\hat\beta)$
- **P>|t|**: p-value for $H_0: \beta=0$ (under assumptions)
- **[0.025, 0.975]**: 95% confidence interval for $\beta$

You can access these programmatically:

```python
# res.params        # coefficients (pandas Series)
# res.bse           # standard errors
# res.pvalues       # p-values
# res.conf_int()    # confidence intervals
```

### Robust standard errors
Robust SE change uncertainty estimates, not coefficients.

```python
# HC3 (cross-section)
# res_hc3 = res.get_robustcov_results(cov_type='HC3')

# HAC/Newey-West (time series)
# res_hac = res.get_robustcov_results(cov_type='HAC', cov_kwds={'maxlags': 4})
```

Why this matters:
- Cross-sectional data often has **heteroskedasticity** (error variance changes across observations).
- Time series often has **autocorrelation** (errors correlated over time).

### Practical rule
- Use sklearn for predictive pipelines and cross-validation.
- Use statsmodels when you need inference and careful interpretation.


<a id="rolling-regression"></a>
## Rolling regression

### Goal
Fit the same regression repeatedly on a moving window.

We will start with a simple model:
- GDP growth ~ yield curve spread (lagged)



### Your Turn (1): Load data and set up the window


In [ ]:
import pandas as pd

path = PROCESSED_DIR / 'macro_quarterly.csv'
if path.exists():
    df = pd.read_csv(path, index_col=0, parse_dates=True)
else:
    df = pd.read_csv(SAMPLE_DIR / 'macro_quarterly_sample.csv', index_col=0, parse_dates=True)

y_col = 'gdp_growth_qoq'
x_col = 'T10Y2Y_lag1'

df_m = df[[y_col, x_col, 'recession']].dropna().copy()

# Rolling window length in quarters
window = 40  # ~10 years
df_m.head()



### Your Turn (2): Fit rolling windows and collect coefficients


In [ ]:
import numpy as np
import statsmodels.api as sm

rows = []
for end in range(window, len(df_m) + 1):
    chunk = df_m.iloc[end - window : end]
    X = sm.add_constant(chunk[[x_col]], has_constant='add')
    y = chunk[y_col]
    res = sm.OLS(y, X).fit()

    # Record the coefficient on x_col and a simple CI
    beta = float(res.params[x_col])
    ci_low, ci_high = res.conf_int().loc[x_col].tolist()
    rows.append({
        'date': chunk.index.max(),
        'beta': beta,
        'ci_low': float(ci_low),
        'ci_high': float(ci_high),
    })

roll = pd.DataFrame(rows).set_index('date')
roll.head()



<a id="coefficient-drift"></a>
## Coefficient drift

### Goal
Visualize coefficient stability over time.



### Your Turn (1): Plot coefficient + CI over time


In [ ]:
import matplotlib.pyplot as plt

# TODO: Plot roll['beta'] and a shaded CI band.
...



### Your Turn (2): Summarize coefficient distribution


In [ ]:
# TODO: Compute summary stats for beta.
# Identify periods where the sign changed.
roll['beta'].describe()



<a id="regime-interpretation"></a>
## Regime interpretation

### Goal
Compare coefficient drift to recession periods.

This is not proof of causality.
It is a structured way to ask: "does the relationship change during recessions or different eras?"



### Your Turn (1): Overlay recession shading (simple)


In [ ]:
# TODO: Create a recession indicator aligned to roll index.
# Hint: use df_m['recession'] reindexed to roll.index
...



### Reflection


- When does the sign or magnitude change?
- What macro regimes might explain it?
- If you were building a model, would you trust one fixed coefficient?



<a id="checkpoint-self-check"></a>
## Checkpoint (Self-Check)
Run a few asserts and write 2-3 sentences summarizing what you verified.



In [ ]:
# TODO: After you build X/y and split by time, validate the split.
# Example (adjust variable names):
# assert X_train.index.max() < X_test.index.min()
# assert y_train.index.equals(X_train.index)
# assert y_test.index.equals(X_test.index)
# assert not X_train.isna().any().any()
# assert not X_test.isna().any().any()
...



## Extensions (Optional)
- Try one additional variant beyond the main path (different features, different split, different model).
- Write down what improved, what got worse, and your hypothesis for why.



## Reflection
- What did you assume implicitly (about timing, availability, stationarity, or costs)?
- If you had to ship this model, what would you monitor?



<a id="solutions-reference"></a>
## Solutions (Reference)

Try the TODOs first. Use these only to unblock yourself or to compare approaches.

<details><summary>Solution: Rolling regression</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 06_rolling_regressions_stability — Rolling regression
import pandas as pd
import statsmodels.api as sm

df = pd.read_csv(SAMPLE_DIR / 'macro_quarterly_sample.csv', index_col=0, parse_dates=True).dropna()
window = 12  # quarters
betas = []
dates = []
for i in range(window, len(df)+1):
    sub = df.iloc[i-window:i]
    X = sm.add_constant(sub[['T10Y2Y']])
    res = sm.OLS(sub['gdp_growth_qoq'], X).fit()
    betas.append(res.params['T10Y2Y'])
    dates.append(sub.index[-1])
beta_series = pd.Series(betas, index=dates)
beta_series.tail()
```

</details>

<details><summary>Solution: Coefficient drift</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 06_rolling_regressions_stability — Coefficient drift
import matplotlib.pyplot as plt
beta_series.plot(title='Rolling coefficient: GDP growth ~ yield spread')
plt.show()
```

</details>

<details><summary>Solution: Regime interpretation</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 06_rolling_regressions_stability — Regime interpretation
# Coefficient drift suggests relationships are not stable across eras.
```

</details>

